In [ ]:
pip install 'gymnasium[atari]'

In [ ]:
pip install atari-py

In [ ]:
pip install 'gymnasium[accept-rom-license]'

In [ ]:
import torch
import gymnasium as gym
import random
import cv2

In [ ]:
env = gym.make("ALE/SpaceInvaders-v5",render_mode="rgb_array")

In [ ]:
height,width,channel = env.observation_space.shape
actions = env.action_space.n
env.unwrapped.get_action_meanings()

In [ ]:
e = 0.7 ##epsilon
n= random.random()
print(n)

In [ ]:
def pre_process(M):
    
    crop_img=[]
    x=M
    try:
        
        gray = cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)
        gray = cv2.pyrDown(gray)
        return crop_img.append(gray[4:98,0:80])
    
    except:
        
        gray = cv2.cvtColor(x[0], cv2.COLOR_BGR2GRAY)
        gray = cv2.pyrDown(gray)
        return crop_img.append(gray[4:98,0:80])

In [ ]:
gray = cv2.cvtColor(srb[1][3], cv2.COLOR_BGR2GRAY)

In [ ]:
#preprocessing of image function called inside replay buffer
#and to be used everytime before passing into neural net model

#initiate a replay buffer
#Replay buffer stores 10,000 timesteps worth data

from collections import deque
buffer_size = 10000
replay_buffer= deque(maxlen=buffer_size)
done = False
image_state_buffer=[]
state = env.reset()
while len(replay_buffer)< buffer_size:
    
    action = random.choice([0,1,2,3,4,5])
    next_state, reward, done, _,_ = env.step(action)
    pre_process(state)
    pre_process(next_state)
    replay_buffer.append((state,action, reward, next_state, done))
    if done:
        state = env.reset()
        done = False
    else:
        state = next_state
     
            
        

    

In [ ]:
##A function to update the replay buffer
def update_replay(new_buf,replay_buffer):
    arr={}
    for i in range(0,100):
        
        
        try:
            num=random.randint(0,10000)
            while(num in arr.values()):
                num=random.randint(0,10000)
            arr[i]=num
            new_tensor = new_buf[i]
            del replay_buffer[new_tensor]
            replay_buffer.insert(num,new_tensor)
            
        except:
            arr[i]=num
            new_tensor = new_buf[i]
            del replay_buffer[num]
            replay_buffer.insert(num,new_tensor)
            
    return replay_buffer
       

In [ ]:
#Defining a deep neural network model for training using pyTorch
#model for Q-val determination
import torch
from torch import nn
import torch.nn.functional as F
class image_model(nn.Module):
    def __init__(self,actions):
        self.actions = action
        super(image_model,self).__init__()
        self.conv_flat_ANN_stack = nn.Sequential(
                 nn.Conv2d(3,32,8,stride=4),
                 nn.ReLU(),
                 nn.Conv2d(32,64,4,stride=2),
                 nn.ReLU(),
                 nn.Conv2d(64,64,3),
                 nn.Flatten(),
                 nn.Linear(22528,256),
                 nn.ReLU(),
                 nn.Linear(256,actions),
                 nn.ReLU(),
                 nn.Linear(actions,actions)
            )
    def forward(self,x):
            return self.conv_flat_ANN_stack(x)
            

        

In [ ]:
Model = image_model(actions)

In [ ]:
##created 2nd network for finding target values
import copy
def update_Target_model():
    target_model = copy.deepcopy(Model)
    return target_model

In [ ]:
target_net = update_Target_model()

In [ ]:
##Prepare batch to be loaded into Q-learning model and target network 


In [ ]:
##Forward pass next_state from the batch into target network
##Use max function to find maximum output Y


In [ ]:

import numpy as np
##Data loader
#Lets say i am running 10 iterations then for every 1 out 0f 10 iteration there should be a new dataset in data loader
#batch size 100
class loaddat():
    def __init__(self):
        self.l_actions=[]
        self.l_current=[]
        self.l_reward=[]
        self.l_next=[]
        self.batch_size=10
        
    def data_loader(self,replay_buffer):
        self.l_actions=[]
        self.l_current=[]
        self.l_reward=[]
        self.l_next=[]
        
        for i in range(0,self.batch_size):
            x = random.randint(0,len(replay_buffer)-1)
            self.l_current.append(replay_buffer[x][0])
            self.l_actions.append(replay_buffer[x][1])
            self.l_next.append(replay_buffer[x][3])
            self.l_reward.append(replay_buffer[x][2])

        return self.l_current,self.l_actions,self.l_next,self.l_reward
obj1 = loaddat()


In [ ]:
##Forward pass into Q-Net
class forward_Q(image_model):
    def __init__(self):
        super().__init__(actions)
        self.Y=[]
        self.y = torch.zeros(1,6)
        self.q_val=[]
    def q_val_pred(self,dl,aa):
        self.Y=[]
        self.y = torch.zeros(1,6)
        self.q_val=[]
        
        for i in range(0,len(dl)):
            
            try:
                dl[i] = torch.tensor(dl[i],dtype =torch.float32)
                dl[i] = torch.permute(dl[i],(2,0,1)).unsqueeze(0)
            except:
                dl[i] = torch.tensor(dl[i][0],dtype =torch.float32)
                dl[i] = torch.permute(dl[i],(2,0,1)).unsqueeze(0)
                
            self.Y.append(super().forward(dl[i]))
                
        for i in range(0,len(self.Y)):
            self.Y[i]=torch.tensor(self.Y[i])
        for i in range(0,len(self.Y)):
            self.y = torch.cat([self.y,self.Y[i]],0)
            
        self.y = self.y[1:11]
   #predicted q-value from Q-Net     
        for i in range(0,len(aa)):
            self.q_val.append(self.y[i][aa[i]])
        self.q_val=torch.tensor(self.q_val,requires_grad=True)##predicted Q val is ready
        return self.q_val
    
obj2 = forward_Q()

In [ ]:
dl[2]

In [ ]:
##Forward pass next_state from the batch into T-learning model
##choose maximum of network output
#Find target value T by calling comp_targ_val
class forward_T_net(image_model):
    def __init__(self):
        super().__init__(actions)
        self.gamma=0.8
    
    def comp_targ_val(self,reward,max_q):
        return reward+(self.gamma*max_q)
    
    def target(self,ndl,r,target_net):
        self.T_output=[]
        for i in range(0,len(ndl)):
         
                ndl[i] = torch.tensor(ndl[i],dtype =torch.float32)
                ndl[i] = torch.permute(ndl[i],(2,0,1)).unsqueeze(0)
                self.T_output.append(target_net.forward(ndl[i]))
    
        self.T=[]
        for i in range(0,len(self.T_output)):
            self.T.append(torch.max(self.T_output[i]))
        self.T=torch.tensor(self.T)##targeted value from Target network
        self.T_val =[]
        for i in range(0,len(self.T)):
            self.T_val.append(self.comp_targ_val(r[i],self.T[i]))
        self.T_val = torch.tensor(self.T_val,requires_grad=True)##Now Targeted value is ready
        return self.T_val
    
    
obj3 = forward_T_net()

In [ ]:
ndl

In [ ]:
##Compute loss and backpropogate loss
##Use MSE loss
import torch.optim as optim
# create your optimizer
loss= nn.MSELoss()
optimizer = optim.SGD(Model.parameters(), lr=0.01)

# in your training loop:
   # Does the update

In [ ]:
##initial Main loop for training Q-Net
iterations = 10000
for i in range(0,iterations):
    dl,aa,ndl,r = obj1.data_loader(replay_buffer)#call everything at once
    q_val = obj2.q_val_pred(dl,aa)
    if i==0 or i%100==0 :
        
        T_val = obj3.target(ndl,r,target_net)#Forward pass in T-Network
        
    optimizer.zero_grad()   # zero the gradient buffers
    LOSS = loss(q_val,T_val)
    LOSS.backward()
    optimizer.step() #back_propagate(Loss)
    if(i%2000==0):
        print("Loss:{}".format(LOSS))
        

In [ ]:
PATH = "/Users/sidhantdas/Documents/weigths"
torch.save(Model.state_dict(), PATH)

In [ ]:
len(dl)

In [ ]:
Model.state_dict()

In [ ]:
#Post training
srb = deque()
srb.clear()
rb = deque(maxlen =10000)
rb.clear()
episodes = 10000
new_buf = deque(maxlen = 100)
new_buf.clear()
score=0
for game in range(0,episodes):
    state= env.reset()
    done=False
    ctr=0
    err_ctr = 0
    while not done: 
                state = env.render()
                if n>e or score==0 :
                    action = random.choice([0,1,2,3,4,5])
                else:
                    a_state = torch.tensor(state,dtype =torch.float32)
                    a_state = torch.permute(a_state,(2,0,1)).unsqueeze(0)
                    output = Model.forward(a_state)
                    action = np.argmax(output.detach().numpy)
                n_state, reward, done, cond, info = env.step(action)
                score+=reward
                try:
                    srb.append((state,action,reward,n_state))
                except:
                    srb.append((state.numpy(),action,reward,n_state))  
                    state =n_state
                try:
                    next_state=pre_process(srb[ctr][3])
                    state =pre_process(srb[ctr][0])
                    if len(new_buf) < 100: 
                                       new_buf.append((state,action, reward, next_state, done))
                                       
                        
                except:
                    next_state=pre_process(srb[ctr][3][0])
                    state =pre_process(srb[ctr][0][0])
                    if len(new_buf) < 100:
                                       new_buf.append((state,action, reward, next_state, done))
                print("Episode:{},Score:{},Action:{} ".format(game,score,action))
                ctr+=1
                    

                if(game%10==0 and game>0):
                    #update replay buffer
                    iterations = 10000
                    rb = update_replay(new_buf,replay_buffer)
                    for i in range(0,iterations):
                    
                    #Call in data_loader
                        dl,aa,ndl,r=obj1.data_loader(rb)
                    #call To train the Q-Net and T-net
                        q_val = obj2.q_val_pred(dl,aa)
                        if i==0 or i%100==0 :
                            T_val = obj3.target(ndl,r,target_net)#Forward pass in T-Network
                        optimizer.zero_grad()   # zero the gradient buffers
                        LOSS = loss(q_val,T_val)
                        LOSS.backward()
                        optimizer.step() #back_propagate(Loss)
                        if(game%20000==0):
                            print("episode:{},Loss:{}".format(episode,LOSS))
                
